In [1]:
import mxnet as mx

from mxnet.ndarray import linalg_gemm as gemm
from mxnet.ndarray import linalg_gemm2 as gemm2
from mxnet.ndarray import linalg_potrf as potrf
from mxnet.ndarray import linalg_trsm as trsm
from mxnet.ndarray import linalg_sumlogdiag as sumlogdiag

import mxnet.ndarray as nd

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.figsize"] = (10, 5)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-6142d417481a>", line 2, in <module>
    import matplotlib.pyplot as plt
  File "/opt/conda/lib/python3.7/site-packages/matplotlib/__init__.py", line 138, in <module>
    from . import cbook, rcsetup
  File "/opt/conda/lib/python3.7/site-packages/matplotlib/rcsetup.py", line 24, in <module>
    from matplotlib.fontconfig_pattern import parse_fontconfig_pattern
  File "/opt/conda/lib/python3.7/site-packages/matplotlib/fontconfig_pattern.py", line 18, in <module>
    from pyparsing import (Literal, ZeroOrMore, Optional, Regex, StringEnd,
  File "/opt/conda/lib/python3.7/site-packages/pyparsing.py", line 6366, in <module>
    class pyparsing_common:
  File "/opt/conda/lib/python3.7/site-packages/pyparsing.py", line 6524, in pyparsing_common
    hex_integer = Word(hexnums).se

KeyboardInterrupt: 

In [ ]:

alpha = 0.5
sigma = 0.5
theta = np.pi / 6
T = 50

A = nd.array([[np.cos(theta), -np.sin(theta)],
              [np.sin(theta), np.cos(theta)]])
B = nd.array([[1, 0]])

S_h = nd.array(np.square(alpha) * np.eye(2))
S_v = nd.array(np.square(sigma) * np.eye(1))

v = []
# initial state h_0
h = np.array([1, 0])
for t in range(T):
    # h_t = Bh_{t-1} + \epsilon_h
    h = np.random.multivariate_normal(A.asnumpy().dot(h), S_h.asnumpy())

    # v_t = Ah_t + \epsilon_v
    vv = np.random.normal(B.asnumpy().dot(h), S_v.asnumpy())

    v.append(vv)
v = nd.array(np.array(v).reshape((T,1)))

In [ ]:
plt.plot(v.asnumpy());

In [ ]:
def LDS_forward(v, A, B, S_h, S_v):

    H = A.shape[0] # dim of latent state
    D = B.shape[0] # dim of observation
    T = v.shape[0] # num of observations

    f_0 = nd.zeros((H,1))
    F_0 = nd.zeros((H,H))

    eye_h = nd.array(np.eye(H))

    F_t = None
    f_t = None
    F_seq = []
    f_seq = []
    log_p_seq = []

    for t in range(T):

        if t == 0:
            # At the first time step, use the prior
            mu_h = f_0
            S_hh = F_0
        else:
            # Otherwise compute using update eqns.
            mu_h = gemm2(A, f_t)
            S_hh = gemm2(A, gemm2(F_t, A, transpose_b=1)) + S_h

        # direct transcription of the update equations above
        mu_v = gemm2(B, mu_h)
        S_hh_x_B_t = gemm2(S_hh, B, transpose_b=1)
        S_vv = gemm2(B, S_hh_x_B_t) + S_v
        S_vh = gemm2(B, S_hh)

        # use potrf to compute the Cholesky decomposition S_vv = LL^T
        S_vv_chol = potrf(S_vv)

        # K = S_hh X with X = B^T S_vv^{-1}
        # We have X = B^T S_vv^{-1} => X S_vv = B^T => X LL^T = B^T
        # We can thus obtain X by solving two linear systems involving L
        K = trsm(S_vv_chol, trsm(S_vv_chol, S_hh_x_B_t, rightside=1, transpose=1), rightside=1)

        delta = v[t] - mu_v
        f_t = mu_h + gemm2(K, delta)

        ImKB = eye_h - gemm2(K, B)
        F_t = (gemm2(ImKB, gemm2(S_hh, ImKB, transpose_b=True))
                       + gemm2(K, gemm2(S_v, K, transpose_b=True), name="Ft"))

        # save filtered covariance and mean
        F_seq.append(F_t)
        f_seq.append(f_t)

        # compute the likelihood using mu_v and L (LL^T = S_vv)
        Z = trsm(S_vv_chol, trsm(S_vv_chol, delta), transpose=1)
        log_p = (-0.5 * (mx.nd.reshape(gemm2(delta, Z, transpose_a=True), shape=(0,), name="reshaped")
                         + D*np.log(2.0 * np.pi)) - sumlogdiag(S_vv_chol))
        log_p_seq.append(log_p)


    return f_seq, F_seq, log_p_seq

In [ ]:
A = nd.array([[4.0, 1.0], [1.0, 4.25]])
B = potrf(A)
# potrf(A) = [[[2.0, 0], [0.5, 2.0]], [[4.0, 0], [1.0, 4.0]]]
B

In [ ]:
gemm2(B, B,transpose_b=True)

In [ ]:
A

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (12, 5)

In [ ]:
df = pd.read_csv("https://datahub.io/core/bond-yields-us-10y/r/monthly.csv", header=0)

In [ ]:
df.set_index("Date")

# get the time series
ts = df.values[:,1]

# Let us normalize the time series
ts = np.array((ts - np.mean(ts)) / np.std(ts), dtype=np.double)

In [20]:
import torch
torch.randn(1, 3)

tensor([[ 1.8596, -1.2841, -1.3015]])

In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [24]:
hidden = (torch.randn(1, 1, 3),
          torch.randn(1, 1, 3))
hidden

(tensor([[[-0.1955, -0.9656,  0.4224]]]),
 tensor([[[ 0.2673, -0.4212, -0.5107]]]))

In [ ]:
plt.plot(ts);

In [4]:
from __future__ import print_function
import mxnet as mx
from mxnet import nd, autograd
import numpy as np
mx.random.seed(1)
ctx = mx.gpu(0)

In [ ]:
with open("../data/nlp/timemachine.txt") as f:
    time_machine = f.read()
time_machine = time_machine[:-38083]

In [14]:
layer = mx.gluon.rnn.LSTM(100, 1)
layer.initialize()
input = mx.nd.random.uniform(shape=(1, 3, 5))
# by default zeros are used as begin state
output = layer(input)
# manually specify begin state.
# h0 = mx.nd.random.uniform(shape=(3, 3, 100))
# c0 = mx.nd.random.uniform(shape=(3, 3, 100))
# output, hn = layer(input, [h0, c0])

In [15]:
input


[[[0.88996416 0.2945612  0.9355159  0.82150155 0.06717816]
  [0.22636814 0.7380694  0.7463265  0.99051315 0.1059881 ]
  [0.15745817 0.9617902  0.60287184 0.6584172  0.2708346 ]]]
<NDArray 1x3x5 @cpu(0)>

In [16]:
output.shape

(1, 3, 100)

In [19]:
mx.gluon.rnn.LSTM?